# Braced frame modeling

Last updated: 1 Jun, 4:48PM

This script is designed to automate the modelling of a concentric braced frame in Openseespy. The input is currently a csv file that details the design, though it can be integrated with a designer script to intake a design class.

A sample CBF design is saved in a csv file. We first read in the data

In [8]:
import pandas as pd
test_cbf = pd.read_csv('cbf.csv', index_col=0, header=0).T
cbf_params = test_cbf.iloc[0]
# preview the data
cbf_params

num_bays           5.000000
num_stories        4.000000
num_frames         2.000000
S_1                1.185500
T_m                2.749952
k_ratio            5.739507
Q                  0.071810
moat_ampli         1.775365
RI                 1.445491
L_bay             30.000000
h_story           13.000000
S_s                2.281500
W              10687.500000
W_s             8437.500000
mu_1               0.023457
mu_2               0.086961
R_1               12.374762
R_2               51.674927
T_e                2.830994
k_e                0.012748
zeta_e             0.147725
D_m               23.755869
Name: 24, dtype: float64

In [9]:
# temporary fix to add loads to the Series
import numpy as np
cbf_params['w_fl'] = np.array([3., 3., 3., 3., 2.013])
cbf_params['P_lc'] = np.array([1800., 1800., 1800., 1800., 1208.])
cbf_params['column'] = np.array(['W14X873', 'W14X605', 'W14X283', 'W12X58'])
cbf_params['beam'] = np.array(['W36X853', 'W36X853', 'W36X802', 'W36X802'])
cbf_params['brace'] = np.array(['HSS18X18X1/2', 'HSS14X14X5/8', 'HSS16X16X1/2', 'HSS16X16X3/8'])
# accessing a sample value
n_bays = int(cbf_params['num_bays'])
cbf_params

num_bays                                                       5
num_stories                                                    4
num_frames                                                     2
S_1                                                       1.1855
T_m                                                      2.74995
k_ratio                                                  5.73951
Q                                                      0.0718096
moat_ampli                                               1.77536
RI                                                       1.44549
L_bay                                                         30
h_story                                                       13
S_s                                                       2.2815
W                                                        10687.5
W_s                                                       8437.5
mu_1                                                   0.0234575
mu_2                     

# Support utilities

These are utility functions that we can use in the construction of the model later on.

## Descriptions

`bot_gp_coord` is a function that returns the x and z coordinate of a node attached to a bottom node of a brace (e.g. 2103, 2104, 2101, 2102). It takes the node number and figures out its location , then places it appropriately according the bay length and height. It also takes an argument `offset` which is how detached we put the node from its parent node. Here, we leave a default of 0.25, meaning "put the end of the gusset plate 25% of the brace length away from the beam-column joint". IMPORTANT: this function assumes that nodes xxx3 and xxx4 are of a brace going upwards and right (northeast), while xxx1 and xxx2 goes to the northwest. If you have a different numbering system, change the `goes_ne` variable accordingly.

`top_gp_coord` behaves similarly, assuming that nodes xxx1 and xxx5 connects southeast, while xxx2 and xxx6 connects southwest. The logic is handled in the if-else statement, which just checks if the last number is odd or even. Change your line accordingly if you have a different numbering system.

`mid_brace_coord` takes the mid-brace node's number and calculates its position with a camber designed to introduce initial imperfection into the brace. This helps enables its buckling behavior later on. We place the nodes with a default 0.1% camber, meaning the middle node lies 0.1% of the brace length away from the straight line.

IMPORTANT: my system numbers the middle node accordingly to the top node it's attached to. In other words, node 3117 is the  middle node extending southeast from 311. You will have to change this system. For example, for your mid brace node 2106, calculate which is the bottom node it connects to and which is the top node it connects to. Make the appropriate changes in lines `top_node =` and `bot_node = `.

In [10]:
def bot_gp_coord(nd, L_bay, h_story, offset=0.25):
    # from node number, get the parent node it's attached to
    bot_nd = nd//100
    
    # get the bottom node's coordinates
    bot_x_coord = (bot_nd%10)*L_bay
    bot_y_coord = (bot_nd//10 - 1)*h_story
    
    # if last number is 1 or 2, brace connects nw
    # if last number is 3 or 4, brace connects ne
    goes_ne = [3, 4]
    if (nd%10 in goes_ne):
        x_offset = offset/2*L_bay/2
    else:
        x_offset = -offset/2*L_bay/2
    
    y_offset = offset/2 * h_story
    gp_x_coord = bot_x_coord + x_offset
    gp_y_coord = bot_y_coord + y_offset
    
    return(gp_x_coord, gp_y_coord)


def top_gp_coord(nd, L_bay, h_story, offset=0.25):
    # from node number, get the parent node it's attached to
    top_node = nd//100
    
    # extract their corresponding coordinates from the node numbers
    top_x_coord = (top_node%10 + 0.5)*L_bay
    top_y_coord = (top_node//10 - 1)*h_story
    
    # if last number is 1 or 5, brace connects se
    # if last number is 2 or 6, brace connects sw
    if (nd % 10)%2 == 0:
        x_offset = -offset/2*L_bay/2
    else:
        x_offset = offset/2*L_bay/2
    
    y_offset = -offset/2 * h_story
    gp_x_coord = top_x_coord + x_offset
    gp_y_coord = top_y_coord + y_offset
    
    return(gp_x_coord, gp_y_coord)
    

def mid_brace_coord(nd, L_bay, h_story, camber=0.001, offset=0.25):
    # from mid brace number, get the corresponding top and bottom node numbers
    top_node = nd//100
    
    # extract their corresponding coordinates from the node numbers
    top_x_coord = (top_node%10 + 0.5)*L_bay
    top_y_coord = (top_node//10 - 1)*h_story
    
    # if the last number is 8, the brace connects sw
    # if the last number is 7, the brace connects se
    
    if (nd % 10)%2 == 0:
        bot_node = top_node - 10
        x_offset = offset/2 * L_bay/2
    else:
        bot_node = top_node - 9
        x_offset = - offset/2 * L_bay/2
    
    # get the bottom node's coordinates
    bot_x_coord = (bot_node%10)*L_bay
    bot_y_coord = (bot_node//10 - 1)*h_story
    
    # effective length is 90% of the diagonal (gusset plate offset)
    br_x = abs(top_x_coord - bot_x_coord)
    br_y = abs(top_y_coord - bot_y_coord)
    L_eff = (1-offset)*(br_x**2 + br_y**2)**0.5
    
    # angle from horizontal up to brace vector
    from math import atan, asin, sin, cos
    theta = atan(h_story/(L_bay/2))
    
    # angle from the brace vector up to camber
    beta = asin(2*camber)
    
    # angle from horizontal up to camber
    gamma  = theta + beta
    
    # origin is bottom node, adjusted for gusset plate
    # offset is the shift (+/-) of the bottom gusset plate
    # terminus is top node, adjusted for gusset plate (gusset placed opposite direction)
    x_origin = bot_x_coord + x_offset
    # x_terminus = top_x_coord - x_offset
    
    y_offset = offset/2 * h_story
    y_origin = bot_y_coord + y_offset
    # y_terminus = top_y_coord - y_offset
    
    mid_x_coord = x_origin + L_eff/2 * cos(gamma)
    mid_y_coord = y_origin + L_eff/2 * sin(gamma)
    
    return(mid_x_coord, mid_y_coord)

def get_shape(shape_name, member, csv_dir='../resource/'):
    import pandas as pd
    
    if member == 'beam':
        shape_db = pd.read_csv(csv_dir+'beamShapes.csv',
                               index_col=None, header=0)
    elif member == 'column':
        shape_db = pd.read_csv(csv_dir+'colShapes.csv',
                               index_col=None, header=0)
    elif member == 'brace':
        shape_db = pd.read_csv(csv_dir+'braceShapes.csv',
                               index_col=None, header=0)  
    shape = shape_db.loc[shape_db['AISC_Manual_Label'] == shape_name]
    return(shape)

# get shape properties
def get_properties(shape):
    Ag      = float(shape.iloc[0]['A'])
    Ix      = float(shape.iloc[0]['Ix'])
    Iy      = float(shape.iloc[0]['Iy'])
    Zx      = float(shape.iloc[0]['Zx'])
    Sx      = float(shape.iloc[0]['Sx'])
    d       = float(shape.iloc[0]['d'])
    bf      = float(shape.iloc[0]['bf'])
    tf      = float(shape.iloc[0]['tf'])
    tw      = float(shape.iloc[0]['tw'])
    return(Ag, Ix, Iy, Zx, Sx, d, bf, tf, tw)

# Building class

It's advantageous to make a class for our data, which allows us to internally store all of the design parameters and objects of the models (nodes, elements) without having to pass long lists of arguments to each function. We can also store all functions related to the building as methods that act on the Building object itself.

For example, the number_nodes method acts on the Building object by taking the Building and finding its number of bays and stories in order to add the lists of nodes and elements to the Building. Later on, we will add model and analyze as additional methods to the Building.

In [11]:
class Building:
    
    #########################################
    # INITIALIZING
    #########################################
    
    # import attributes as building characteristics from pd.Series
    
    # now the Series you see above is converted into a Building object
    # Values in the series are now attributes of the object
    def __init__(self, design):
        for key, value in design.items():
            setattr(self, key, value)
            
    #########################################
    # NUMBERING NODES AND ELEMENTS
    #########################################
    
    # number nodes
    def number_nodes(self):
        
        n_bays = int(self.num_bays)
        n_stories = int(self.num_stories)
        
        # larger than 8 bays is not supported
        assert n_bays < 9
        
        ###### Main node system ######
        # Fixed nodes are 8xx, with xx being sequential from leftmost base
        #   898 and 899 are reserved for wall
        
        # All nodes are numbered xy, with x indicating floor number 
        #   1 at ground, n_stories+1 at roof
        # and y indicating column line
        #   0 at leftmost, n_bays at rightmost
        
        # Leaning column nodes are appended to the right at the same floor (n_bay+1)
        ##############################
        
        # base nodes
        base_id = 800
        base_nodes = [node for node in range(base_id, base_id+n_bays+1)]
        
        # wall nodes
        wall_nodes = [898, 899]
        
        # floor and leaning column nodes
        floor_id = [10*fl for fl in range(1, n_stories+2)]
        nds = [[nd for nd in range (fl, fl+n_bays+1)] for fl in floor_id]
        leaning_nodes = [(fl+n_bays+1) for fl in floor_id]
        
        # flatten list to get all nodes
        floor_nodes = [nd for fl in nds for nd in fl]
            
        # for braced frames, additional nodes are needed
        n_braced = int(round(n_bays/2.25))

        # roughly center braces around middle of the building
        n_start = round(n_bays/2 - n_braced/2)

        # start from first interior bay
        # no ground floor included
        # top brace nodes have numbers xy1, where xy is the main node to its left
        t_brace_id = 1
        brace_tops = [nd*10+t_brace_id for nd in floor_nodes 
                      if ((nd//10)%10 != 1) and
                      (nd%10 >= n_start) and (nd%10 < n_start+n_braced)]

        # bottom brace supports, none on top floor
        # bottom nodes are just a copy of the existing main nodes where braces connect
        brace_bottoms = [nd for nd in floor_nodes
                         if ((nd//10)%10 != n_stories+1) and
                         (nd%10 >= n_start) and (nd%10 <= n_start+n_braced)]

        brace_beam_ends = [nd+10 for nd in brace_bottoms]


        # create two mid-brace nodes for each top brace nodes
        # these are extensions of the support springs (below)
        r_brace_id = 7
        r_brace_nodes = [nd*10+r_brace_id for nd in brace_tops]
        l_brace_id = 8
        l_brace_nodes = [nd*10+l_brace_id for nd in brace_tops]
        brace_mids = l_brace_nodes + r_brace_nodes
            
        
        ###### Spring node system ######
        # Spring support nodes have the coordinates XYA, XY being the parent node
        # A is 6,7,8,9 for S,W,N,E respectively
        
        # Support nodes for braced frames have different systems
        # Starting from inner SW and going clockwise, the numbering is xy1a
        # where xy1 is the brace_top node, and a is the position indicator
        #       xy13    xy1     xy14
        #           xy12    xy11
        #       xy16            xy15
        #   (xy18)                  (xy17)  <- further down midspan
        ################################
        
        br_top_e_inner = [nd*10+1 for nd in brace_tops]
        br_top_w_inner = [nd*10+2 for nd in brace_tops]
        br_top_west = [nd*10+3 for nd in brace_tops]
        br_top_east = [nd*10+4 for nd in brace_tops]
        br_top_e_outer = [nd*10+5 for nd in brace_tops]
        br_top_w_outer = [nd*10+6 for nd in brace_tops]

        br_top_spr = (br_top_e_inner + br_top_w_inner + 
                      br_top_e_outer + br_top_w_outer)

        br_beam_spr = br_top_west + br_top_east 

        br_bot_w_inner = [nd*100+1 for nd in brace_bottoms
                          if (nd%10 != n_start)]

        br_bot_w_outer = [nd*100+2 for nd in brace_bottoms
                          if (nd%10 != n_start)]

        br_bot_e_inner = [nd*100+3 for nd in brace_bottoms
                          if (nd%10 != n_start+n_braced)]
        br_bot_e_outer = [nd*100+4 for nd in brace_bottoms
                          if (nd%10 != n_start+n_braced)]

        br_bot_spr = (br_bot_w_inner + br_bot_w_outer + 
                      br_bot_e_inner + br_bot_e_outer)
            
        # make south node if not on bottom floor
        s_spr = [nd*10+6 for nd in floor_nodes
                 if ((nd//10)%10 != 1)]
        # make west node if not on the leftmost column and bottom floor
        w_spr = [nd*10+7 for nd in floor_nodes
                 if (nd%10) != 0 and ((nd//10)%10 != 1)]
        # make north node if not on top floor
        n_spr = [nd*10+8 for nd in floor_nodes
                 if ((nd//10)%10 != n_stories+1)]
        # make east node if not on rightmost column and bottom floor
        e_spr = [nd*10+9 for nd in floor_nodes
                 if (nd%10) != n_bays and ((nd//10)%10 != 1)]
        
        # add an additional support node for shear tabs in CBFs
        e_tab_spr = [nd*10 for nd in brace_beam_ends
                     if nd%10 != n_start+n_braced]
        w_tab_spr = [nd*10+5 for nd in brace_beam_ends
                     if nd%10 != n_start]
        tab_spr = e_tab_spr + w_tab_spr
            
        # repeat for leaning columns, only N-S
        lc_spr_nodes = [nd*10+6 for nd in leaning_nodes
                        if ((nd//10)%10 != 1)] + [nd*10+8 for nd in leaning_nodes
                                                  if ((nd//10)%10 != n_stories+1)]
        
        spring_nodes = s_spr + w_spr + n_spr + e_spr
        
        # column elements, series 100
        col_id = 100
        # make column if not the top floor
        col_elems = [nd+col_id for nd in floor_nodes 
                     if ((nd//10)%10 != n_stories+1)]
        
        # leaning column elements 
        lc_elems = [nd+col_id for nd in leaning_nodes 
                     if ((nd//10)%10 != n_stories+1)]
        
        # beam elements, series 200
        beam_id = 200
        # make beam if not the last bay and not the bottom floor
        beam_elems = [nd+beam_id for nd in floor_nodes 
                     if (nd%10 != n_bays) and ((nd//10)%10 != 1)]
        
        # truss elements, series 300
        truss_id = 300
        # make truss on the last bay for all floors
        truss_elems = [nd+truss_id for nd in floor_nodes 
                       if (nd%10 == n_bays)]
        
        # diaphragm elements, series 400
        diaph_id = 400
        # make diaphragm if not the last bay on the bottom floor
        diaph_elems = [nd+diaph_id for nd in floor_nodes 
                       if ((nd//10)%10 == 1) and (nd%10 != n_bays)]
        
        # isolator elements, series 1000
        isol_id = 1000
        # make isolators above base nodes
        isol_elems = [nd+isol_id for nd in base_nodes]
        
        # spring elements, series 5000
        spring_id = 5000
        spring_elems = [nd+spring_id for nd in spring_nodes]
        lc_spr_elems = [nd+spring_id for nd in lc_spr_nodes]
        
        # wall elements, series 8000
        wall_id = 8000
        wall_elems = [nd+wall_id for nd in wall_nodes]
        
        # brace springs, springs 50000, actual brace 900
        # braces are numbered by 9xxxx, where xxxx is the support node belonging
        # to the top/bottom nodes of the brace
        brace_spr_id = 50000
        brace_top_elems = [brace_spr_id+nd for nd in br_top_spr]
        brace_bot_elems = [brace_spr_id+nd for nd in br_bot_spr]
        br_beam_spr_elems = [brace_spr_id+nd for nd in br_beam_spr]

        brace_id = 90000
        brace_end_nodes = (br_top_w_outer + br_top_e_outer + 
                           br_bot_w_outer + br_bot_e_outer)

        brace_bot_end_nodes = (br_bot_w_outer + br_bot_e_outer)

        brace_elems = [brace_id + nd for nd in brace_end_nodes]
        brace_ghost_elems = [brace_id + nd + 5 for nd in brace_bot_end_nodes]

        # brace beams are 2xxx, where xxx is either 0xy for the left parent xy
        # or xy1 for the mid-bay parent xy1
        brace_beams_id = 2000
        br_east_elems = [brace_beams_id+nd for nd in brace_tops]
        br_west_elems = [brace_beams_id+(nd//10) for nd in brace_tops]
        brace_beam_elems = br_east_elems + br_west_elems
        
        self.node_tags = {
            'base': base_nodes,
            'wall': wall_nodes,
            'floor': floor_nodes,
            'leaning': leaning_nodes,
            'spring': spring_nodes,
            'lc_spring': lc_spr_nodes,
            'brace_top': brace_tops,
            'brace_mid': brace_mids,
            'brace_bottom': brace_bottoms,
            'brace_beam_spring': br_beam_spr,
            'brace_top_spring': br_top_spr,
            'brace_bottom_spring': br_bot_spr,
            'brace_beam_end': brace_beam_ends,
            'brace_beam_tab': tab_spr
            }
        
        self.elem_tags = {
            'col': col_elems, 
            'leaning': lc_elems, 
            'beam': beam_elems,
            'truss': truss_elems, 
            'diaphragm': diaph_elems, 
            'isolator': isol_elems, 
            'spring': spring_elems, 
            'lc_spring': lc_spr_elems, 
            'wall': wall_elems,
            'brace': brace_elems,
            'brace_top_springs': brace_top_elems,
            'brace_bot_springs': brace_bot_elems,
            'brace_beam_springs': br_beam_spr_elems,
            'brace_beams': brace_beam_elems,
            'brace_ghosts': brace_ghost_elems
            }
            
        self.elem_ids = {
            'col': col_id, 
            'leaning': col_id, 
            'beam': beam_id,
            'truss': truss_id, 
            'diaphragm': diaph_id, 
            'isolator': isol_id, 
            'spring': spring_id, 
            'lc_spring': spring_id, 
            'wall': wall_id,
            'base': base_id,
            'brace': brace_id,
            'brace_spring': brace_spr_id,
            'brace_beam': brace_beams_id
            }
            
        
    #########################################
    # BUILDING THE MODEL IN OPENSEES
    #########################################
    def model_braced_frame(self):
        # import OpenSees and libraries
        import openseespy.opensees as ops
        
        # remove existing model
        ops.wipe()

        # units: in, kip, s
        # dimensions
        inch    = 1.0
        ft      = 12.0*inch
        sec     = 1.0
        g       = 386.4*inch/(sec**2)
        kip     = 1.0
        ksi     = kip/(inch**2)
        negligible = 1e-15
        
        L_bay = self.L_bay * 12     # ft to in
        h_story = self.h_story * ft
#         w_cases = self.all_w_cases
#         plc_cases = self.all_Plc_cases
        
#         w_floor = w_cases['1.0D+0.5L'] / ft
#         p_lc = plc_cases['1.0D+0.5L'] / ft
        w_floor = self.w_fl / ft    # kip/ft to kip/in
        p_lc = self.P_lc
        
        # set modelbuilder
        # x = horizontal, y = in-plane, z = vertical
        # command: model('basic', '-ndm', ndm, '-ndf', ndf=ndm*(ndm+1)/2)
        ops.model('basic', '-ndm', 3, '-ndf', 6)
        
        # model gravity masses corresponding to the frame placed on building edge
        import numpy as np
        m_grav_inner = w_floor * L_bay / g
        m_grav_outer = w_floor * L_bay / 2 /g
        m_grav_brace = w_floor * L_bay / 2 /g
        m_grav_br_col = w_floor * L_bay * 3/4 /g
        m_lc = p_lc / g
#         # prepend mass onto the "ground" level
#         m_grav_brace = np.insert(m_grav_brace, 0 , m_grav_brace[0])
#         m_grav_inner = np.insert(m_grav_inner, 0 , m_grav_inner[0])
#         m_grav_outer = np.insert(m_grav_outer, 0 , m_grav_outer[0])
#         m_grav_br_col = np.insert(m_grav_br_col, 0 , m_grav_br_col[0])
#         m_lc = np.insert(m_lc, 0 , m_lc[0])
        
        # load for isolators vertical
        p_outer = sum(w_floor)*L_bay/2
        p_inner = sum(w_floor)*L_bay
        
        # nominal change
        L_beam = L_bay
        L_col = h_story
        
        self.number_nodes()
        
        # get column and beam properties
        col_list = self.column
        sample_column = get_shape(col_list[0], 'column')
        
        beam_list = self.beam
        sample_beam = get_shape(beam_list[0], 'beam')
        (Ag_beam, Iz_beam, Iy_beam,
         Zx_beam, Sx_beam, d_beam,
         bf_beam, tf_beam, tw_beam) = get_properties(sample_beam)
        
        
        (Ag_col, Iz_col, Iy_col,
         Zx_col, Sx_col, d_col,
         bf_col, tf_col, tw_col) = get_properties(sample_column)
    
        ############ place nodes #############
        
        # base nodes
        base_nodes = self.node_tags['base']
        
        # place base nodes here
        for b in range(len(base_nodes)):
            ops.node(base_nodes[b], b*L_bay*ft, 0, -1.0*ft)
            ops.fix(base_nodes[b], 1, 1, 1, 1, 1, 1)
    
        
        # TODO: edit the ops.fix conditions for nodes. Only walls and bases should be fully fixed.
        # remember to fix the nodes with
        # ops.fix(node_tag, 1, 1, 1, 1, 1, 1) for fully fixed
        # ops.fix(nd, 0, 1, 0, 1, 0, 1) for fully free
        
        # wall nodes (should only be two)
        n_bays = int(self.num_bays)
        n_floors = int(self.num_stories)
        
        # place wall nodes here
        wall_nodes = self.node_tags['wall']
        ops.node(wall_nodes[0], 0.0*ft, 0.0*ft, 0.0*ft)
        ops.fix(wall_nodes[0], 1, 1, 1, 1, 1, 1)
        ops.node(wall_nodes[1], n_bays*L_beam, 0.0*ft, 0.0*ft)
        ops.fix(wall_nodes[1], 1, 1, 1, 1, 1, 1)
        
        # place main nodes here
        # structure nodes
        # TODO: add mass onto the floor nodes
        # The m_grav_inner and m_grav_outer arrays have masses of the nodes according to their floors.
        # e.g. m_grav_inner[0] is the ground level mass for an interior node of the building, [1] is the first floor above etc
        # Use the command ops.mass(nd, m_nd, m_nd, m_nd, 1e-15, 1e-15, 1e-15) to add the mass
        floor_nodes = self.node_tags['floor']
        brace_bot_nodes = self.node_tags['brace_bottom']
        brace_top_nodes = self.node_tags['brace_top']
        for nd in floor_nodes:
            ops.node(nd, (nd%10)*L_beam*ft, 0.0*ft, ((nd//10)%10-1)*h_story*ft)
            ops.fix(nd, 1, 1, 1, 1, 1, 1)
            # your mass code here!
            # negligible=1e-15
            # ops.mass(nd, m?, m?, m?, negligible, negligible, negligible)
            
        # leaning column nodes - TODO: only fix bottom, add if statement
        leaning_nodes = self.node_tags['leaning']
        for l in range(len(leaning_nodes)):
            ops.node(leaning_nodes[l], (n_bays+1)*L_bay*ft, 0, l*h_story*ft)
            ops.fix(leaning_nodes[l], 0, 1, 1, 1, 0, 1)
            # your mass code here!
            # negligible=1e-15
            # ops.mass(nd, m?, m?, m?, negligible, negligible, negligible)
            
        # SPECIAL CASE: we roller fix the bottom node of the leaning column
        # use ops.fix(nd, 0, 1, 1, 1, 0, 1)
        
        # brace nodes
        ofs = 0.25
        L_diag = ((L_bay/2)**2 + L_col**2)**(0.5)
        # L_eff = (1-ofs) * L_diag
        L_gp = ofs/2 * L_diag
        brace_top_nodes = self.node_tags['brace_top']
        for nd in brace_top_nodes:
            parent_node = nd // 10
            
            # extract their corresponding coordinates from the node numbers
            fl = parent_node//10 - 1
            x_coord = (parent_node%10 + 0.5)*L_beam
            z_coord = fl*L_col
            
            m_nd = m_grav_brace[fl]
            ops.node(nd, x_coord, 0.0*ft, z_coord)
            ops.mass(nd, m_nd, m_nd, m_nd,
                     negligible, negligible, negligible)
            
        # mid brace node adjusted to have camber of 0.1% L_eff
        # L_eff is defined as L_diag - offset
        brace_mid_nodes = self.node_tags['brace_mid']
        for nd in brace_mid_nodes:
            
            # values returned are already in inches
            x_coord, z_coord = mid_brace_coord(nd, L_beam, L_col, offset=ofs)
            ops.node(nd, x_coord, 0.0*ft, z_coord)
        
        
        # spring nodes
        spring_nodes = self.node_tags['spring']
        brace_beam_ends = self.node_tags['brace_beam_end']    
        brace_beam_tab_nodes = self.node_tags['brace_beam_tab']
        
        col_brace_bay_node = [nd for nd in spring_nodes
                              if (nd//10 in brace_beam_ends 
                                  or nd//10 in brace_bot_nodes)
                              and (nd%10 == 6 or nd%10 == 8)]
        
        beam_brace_bay_node = [nd//10*10+9 if nd%10 == 0
                               else nd//10*10+7 for nd in brace_beam_tab_nodes]
        
        grav_spring_nodes = [nd for nd in spring_nodes
                             if (nd not in col_brace_bay_node)
                             and (nd not in beam_brace_bay_node)]
        
        grav_beam_spring_nodes = [nd for nd in grav_spring_nodes
                                  if nd%2 == 1]
        
        grav_col_spring_nodes = [nd for nd in grav_spring_nodes
                                  if nd%2 == 0]
        
        for nd in spring_nodes:
            parent_nd = nd//10
            
            # get multiplier for location from node number
            bay = parent_nd%10
            fl = parent_nd//10 - 1
            
            # "springs" inside the brace frames should be treated differently
            # if it's a column spring, the offset should be dbeam/2 if it's below the column node
            # if it's above the column node, there is a GP node attached to it
            # roughly, we put it 1.2x L_gp, where L_gp is the diagonal offset of the gusset plate
            
            if nd in col_brace_bay_node:
                if nd%10 == 6:
                    y_offset = d_beam/2
                    ops.node(nd, bay*L_beam, 0.0*ft, fl*L_col-y_offset)
                else:
                    y_offset = 1.2*L_gp
                    ops.node(nd, bay*L_beam, 0.0*ft, fl*L_col+y_offset)
                    
            # if it's a beam spring, place it +/- d_col to the right/left of the column node
            elif nd in beam_brace_bay_node:
                x_offset = d_col/2
                if nd%10 == 7:
                    ops.node(nd, bay*L_beam-x_offset, 0.0*ft, fl*L_col) 
                else:
                    ops.node(nd, bay*L_beam+x_offset, 0.0*ft, fl*L_col)
                    
            # otherwise, it is a gravity frame node and can just overlap the main node
            else:
                ops.node(nd, bay*L_beam, 0.0*ft, fl*L_col)
            
        lc_spr_nodes = self.node_tags['lc_spring']
        for nd in lc_spr_nodes:
            parent_nd = nd//10
            
            # get multiplier for location from node number
            bay = parent_nd%10
            fl = parent_nd//10 - 1
            ops.node(nd, bay*L_beam, 0.0*ft, fl*L_col)
            
        brace_beam_spr_nodes = self.node_tags['brace_beam_spring']
        for nd in brace_beam_spr_nodes:
            grandparent_nd = nd//100
            
            # extract their corresponding coordinates from the node numbers
            x_offset = 1.2*L_gp
            fl = grandparent_nd//10 - 1
            x_coord = (grandparent_nd%10 + 0.5)*L_beam
            z_coord = fl*L_col
            
            # place the node with the offset l/r of midpoint according to suffix
            if nd%10 == 3:
                ops.node(nd, x_coord-x_offset, 0.0*ft, z_coord)
            else:
                ops.node(nd, x_coord+x_offset, 0.0*ft, z_coord)
            
        for nd in brace_beam_tab_nodes:
            parent_nd = nd//10
            
            # get multiplier for location from node number
            bay = parent_nd%10
            fl = parent_nd//10 - 1
            
            x_offset = d_col/2
            if nd%10 == 5:
                ops.node(nd, bay*L_beam-x_offset, 0.0*ft, fl*L_col) 
            else:
                ops.node(nd, bay*L_beam+x_offset, 0.0*ft, fl*L_col)
        
        # each end has offset/2*L_diagonal assigned to gusset plate offset
        brace_bot_gp_nodes = self.node_tags['brace_bottom_spring']
        for nd in brace_bot_gp_nodes:
            
            # values returned are already in inches
            x_coord, z_coord = bot_gp_coord(nd, L_beam, L_col, offset=ofs)
            ops.node(nd, x_coord, 0.0*ft, z_coord)
        
        brace_top_gp_nodes = self.node_tags['brace_top_spring']
        for nd in brace_top_gp_nodes:
            # values returned are already in inches
            x_coord, z_coord = top_gp_coord(nd, L_beam, L_col, offset=ofs)
            ops.node(nd, x_coord, 0.0*ft, z_coord)
        
        print('Nodes placed.')
        
################################################################################
# define materials
################################################################################

        # define your tags here. Tags are numbers for materials. There is no system, except to not reuse a tag.
    
        # define material: steel
        Es  = 29000*ksi     # initial elastic tangent
        nu  = 0.2          # Poisson's ratio
        Gs  = Es/(1 + nu) # Torsional stiffness modulus
        J   = 1e10          # Set large torsional stiffness
    
        # TODO: make an elastic stiff material
        # Make an elastic material with the stiffness of steel (Es)
        # https://openseespydoc.readthedocs.io/en/latest/src/ElasticUni.html
        # your code here. 
        
        # TODO: make an elastic "ghost" material with very small stiffness (100.0 ksi)
        # https://openseespydoc.readthedocs.io/en/latest/src/ElasticUni.html
        # your code here. 
        
        # TODO: make an elastic material in the torsion direction with stiffness J
        # your code here
        
        # TODO: make a Steel02 material with the following properties
        # https://openseespydoc.readthedocs.io/en/latest/src/steel02.html
        Fy  = 50*ksi        # yield strength
        b   = 0.1           # hardening ratio
        R0 = 15
        cR1 = 0.925
        cR2 = 0.15
        # your code here
        
        # TODO: 'wrap' your steel material with a fatigue material to allow it to weaken after repeated cycling
        # https://openseespydoc.readthedocs.io/en/latest/src/Fatigue.html
        # ops.uniaxialMaterial('Fatigue', your_new_material_tag, your_steel02_material_tag)
        
        # Gusset plate 
        W_w = (L_gp**2 + L_gp**2)**0.5
        L_avg = 0.75* L_gp
        t_gp = 1.375*inch
        Fy_gp = 50*ksi
        
        My_GP = (W_w*t_gp**2/6)*Fy_gp
        K_rot_GP = Es/L_avg * (W_w*t_gp**3/12)
        b_GP = 0.01
        
        # TODO: make a Steel02 material with rotational properties of a gusset plate.
        # It has an 'Fy' strength of My_GP, an 'E' stiffness of value K_rot_GP, a post-yielding ratio of b_GP, 
        # and the remaining parameters is the same as your steel
        # your code here

## Test out the building

First, initialize it by getting the parameters from the Series. Then number the nodes.

In [12]:
cbf_bldg = Building(cbf_params)

cbf_bldg.number_nodes()

cbf_bldg.node_tags
cbf_bldg.elem_ids


{'col': 100,
 'leaning': 100,
 'beam': 200,
 'truss': 300,
 'diaphragm': 400,
 'isolator': 1000,
 'spring': 5000,
 'lc_spring': 5000,
 'wall': 8000,
 'base': 800,
 'brace': 90000,
 'brace_spring': 50000,
 'brace_beam': 2000}

## Try modeling the frame

We start with just the nodes for now.

In [13]:
cbf_bldg.model_braced_frame()

Nodes placed.
